# Data mining seminar project

In [2]:
from datetime import date

import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import get_scorer
from imblearn.over_sampling import SMOTE

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from scipy.stats import uniform, loguniform, randint

## Data preparation

- load datasets
- handle MaritalStatus
	- only M (Maried) ann S (Single), NaNs most common, treated as value (not missing) for people in relationship and not married, divorced, or widowed
- correct number of children (cannot be negative)
- add has children column
- add income per dependant column (1 + num of children)
- encode responded to boolean
- handle date format for transaction and get age
- aggregate transaction data
	- minimum age - number of days since last transaction
	- number of transaction (count) total, 
	- monetary value (amount) total, average
- combine data
- encode data to numeric for further tasks

In [3]:
cust = pd.read_csv('data/customers.csv').set_index('CardID')
tran = pd.read_csv('data/transactions.csv').set_index('CardID')
camp = pd.read_csv('data/campaign.csv').set_index('CardID')

cust.replace({ 'MaritalStatus': { 'M': 'Married', 'S': 'Single'}}, inplace=True)
cust.fillna({ 'MaritalStatus': 'Other' }, inplace=True)

cust['NumChildren'] = cust['NumChildren'].clip(0, None)
cust['HasChildren'] = cust['NumChildren'] > 0
cust['HHIncodePerDependant'] = cust['HHIncomeMed']/(1 + cust['NumChildren'])

camp['Responded'] = camp['Responded'] == 'T'

tran['Date'] = tran['Date'].astype(str)
tran['Date'] = tran['Date'].transform(
	lambda x: date(year=int(x[:4]), month=int(x[4:6]), day=int(x[6:])))

tran['Age'] = tran['Date'].max() - tran['Date']
tran['Age'] = tran['Age'].transform(lambda x: x.days)

tran_agg = tran.groupby('CardID').apply(func=lambda gr: pd.Series({
	'age_min' : gr.Age.min(),
	'count_total' : gr.shape[0],
	# 'count_180' : gr[gr.Age < 180].shape[0],
	# 'count_90' : gr[gr.Age < 90].shape[0],
	# 'count_30' : gr[gr.Age < 30].shape[0],
	'amount_total' : gr.Amount.sum(),
	# 'amount_180' : gr[gr.Age < 180].Amount.sum(),
	# 'amount_90' : gr[gr.Age < 90].Amount.sum(),
	# 'amount_30' : gr[gr.Age < 30].Amount.sum(),
	'amount_avg' : gr.Amount.mean(),
}))

# for days in [90, 30]:
# 	tran_agg[f'used_{days}'] = tran_agg[f'count_{days}'] > 0


data = cust.join(tran_agg.add_prefix('Tran_')).join(camp)

data

,MaritalStatus,NumChildren,LoS,HHIncomeMed,HasChildren,HHIncodePerDependant,Tran_age_min,Tran_count_total,Tran_amount_total,Tran_amount_avg,Responded
CardID,,,,,,,,,,,
C0100000199,Single,4,1.156164,71079.744865,True,14215.948973,1.0,3.0,597.00,199.000000,False
C0100000343,Other,1,3.002740,79424.115726,True,39712.057863,114.0,6.0,700.94,116.823333,False
C0100000375,Single,0,0.068493,41878.414258,False,41878.414258,59.0,4.0,223.98,55.995000,False
C0100000482,Single,1,1.356164,62924.588763,True,31462.294381,20.0,4.0,197.98,49.495000,False
C0100000689,Married,3,2.484932,46616.718039,True,11654.179510,4.0,2.0,428.00,214.000000,False
...,...,...,...,...,...,...,...,...,...,...,...
C0106595162,Married,1,1.931507,49841.914121,True,24920.957061,92.0,2.0,388.99,194.495000,False
C0106596136,Married,3,1.400000,88015.076144,True,22003.769036,5.0,2.0,108.99,54.495000,False
C0106596422,Other,0,1.558904,46617.938780,False,46617.938780,134.0,2.0,948.00,474.000000,False


In [4]:
data_enc = pd.get_dummies(data, columns=['MaritalStatus'])
data_enc.drop(['MaritalStatus_Other'], axis=1, inplace=True)

data_enc.insert(0, 'MaritalStatus_Single', data_enc.pop('MaritalStatus_Single'))
data_enc.insert(1, 'MaritalStatus_Married', data_enc.pop('MaritalStatus_Married'))

data_enc

,MaritalStatus_Single,MaritalStatus_Married,NumChildren,LoS,HHIncomeMed,HasChildren,HHIncodePerDependant,Tran_age_min,Tran_count_total,Tran_amount_total,Tran_amount_avg,Responded
CardID,,,,,,,,,,,,
C0100000199,True,False,4,1.156164,71079.744865,True,14215.948973,1.0,3.0,597.00,199.000000,False
C0100000343,False,False,1,3.002740,79424.115726,True,39712.057863,114.0,6.0,700.94,116.823333,False
C0100000375,True,False,0,0.068493,41878.414258,False,41878.414258,59.0,4.0,223.98,55.995000,False
C0100000482,True,False,1,1.356164,62924.588763,True,31462.294381,20.0,4.0,197.98,49.495000,False
C0100000689,False,True,3,2.484932,46616.718039,True,11654.179510,4.0,2.0,428.00,214.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...
C0106595162,False,True,1,1.931507,49841.914121,True,24920.957061,92.0,2.0,388.99,194.495000,False
C0106596136,False,True,3,1.400000,88015.076144,True,22003.769036,5.0,2.0,108.99,54.495000,False
C0106596422,False,False,0,1.558904,46617.938780,False,46617.938780,134.0,2.0,948.00,474.000000,False


## Classification

Object for data representation

- split to test and train
- preprocess data using train split
- get metrics

In [25]:
class Dataset:
	def __init__(self, data, target=None, prep=None, 
			test_size=0.2, random_state=None):
		
		self.data = { 'all': data }
		self.target = target

		self.data['train'], self.data['test'] = train_test_split(
			self.data['all'], stratify=self.y(),
			test_size=test_size, random_state=random_state)

		self.prep = prep
		if self.prep:
			self.prep.fit(self.X('train', use_prep=False))

	def y(self, split='all'):
		return self.data[split][self.target]

	def X(self, split='all', use_prep=True):
		X_ = self.data[split].drop(self.target, axis=1)
		
		if self.prep and use_prep:
			X_ = self.prep.transform(X_)

		return X_

ds = Dataset(data_enc, target='Responded', prep=StandardScaler(), random_state=37)

Object for classifier representation

- oversample data (if selected)
- optimize hyperparameters
- calculate metrics


In [ ]:
class Classifier:
	instances = {}

	def __init__(self, name, clf, param):
		self.name = name
		self.clf = clf
		self.param = param

		Classifier.instances[name] = self
			
	def optimize_param(self, dataset, over_sample=False,
			metric='balanced_accuracy', n_iter=30, random_state=None, verbose=0):
		
		self.opt_dataset = dataset
		self.opt_clf = RandomizedSearchCV(
			self.clf, self.param, scoring=metric, n_jobs=-1, n_iter=n_iter, 
				random_state=random_state, verbose=verbose)
		
		X = dataset.X('train')
		y = dataset.y('train')

		

		self.opt_clf.fit(X, y)
		if over_sample:
			sm = SMOTE(random_state=random_state)
			X_res, y_res = sm.fit_resample(X, y)

		
	def get_metrics(self, dataset=None, metrics=['accuracy', 'balanced_accuracy'], as_string=False):
		if dataset is None:
			dataset = self.opt_dataset

		d = { 'name': self.name }

		for met in metrics:
			scorer = get_scorer(met)
			for split in ['train', 'test']:
				d[f'{met}_{split}'] = float(scorer(self.opt_clf, 
					dataset.X(split), dataset.y(split)))


		self.metrics = d

		if as_string:
			keys = d.keys()
			values = [f'{v:0.3f}' if isinstance(v, float) else v for v in d.values()]
			return ', '.join([f'{k}: {v}' for k, v in zip(keys, values)])

		return d		

classifiers = {
	'nn': {
		'clf': MLPClassifier(),
		'param': {
			'hidden_layer_sizes' : [
				(128,),
				(64, 64),
				(32, 32, 32),
			],
			'solver' : ['adam', 'sgd'],
			'alpha' : loguniform(1e-4, 1e-2),
			'learning_rate' : ['constant', 'invscaling'],
			'learning_rate_init' : loguniform(1e-4, 1e-2),
		}
	}
}

### K-nearest neighbour classifier

In [33]:
knn_clf = Classifier('knn',	
	KNeighborsClassifier(), 
	param={
		'n_neighbors' : randint(3, 10),
		'weights' : ['uniform', 'distance'],
		'algorithm' : ['ball_tree', 'kd_tree'],
		'leaf_size' : randint(10, 50),
		'p' : [1, 2],
	})

knn_clf.optimize_param(ds, random_state=37, verbose=2)
print(knn_clf.get_metrics(as_string=True))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END algorithm=kd_tree, leaf_size=22, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=kd_tree, leaf_size=22, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=kd_tree, leaf_size=22, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=kd_tree, leaf_size=22, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=ball_tree, leaf_size=45, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=kd_tree, leaf_size=22, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=kd_tree, leaf_size=34, n_neighbors=8, p=2, weights=distance; total time=   0.1s
[CV] END algorithm=kd_tree, leaf_size=34, n_neighbors=8, p=2, weights=distance; total time=   0.2s
[CV] END algorithm=ball_tree, leaf_size=45, n_neighbors=6, p=1, weights=distance; total time=   0.2s
[CV] END algorithm=ball_tree, leaf_size=45,

### Random forrest

In [34]:
rf_clf = Classifier('random forest', 
	RandomForestClassifier(class_weight='balanced'),
	param={
		'n_estimators' : randint(10, 50),
		'criterion' : ['gini', 'entropy'],
		'bootstrap' : [True, False],
		'min_samples_leaf' : randint(1, 5),
		'min_samples_split' : randint(2, 10)
	})

rf_clf.optimize_param(ds, random_state=37, verbose=2)
print(rf_clf.get_metrics(ds, as_string=True))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=3, min_samples_split=5, n_estimators=13; total time=   0.5s
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=3, min_samples_split=5, n_estimators=13; total time=   0.5s
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=3, min_samples_split=5, n_estimators=13; total time=   0.5s
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=3, min_samples_split=5, n_estimators=13; total time=   0.5s
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=3, min_samples_split=5, n_estimators=13; total time=   0.5s
[CV] END bootstrap=True, criterion=entropy, min_samples_leaf=4, min_samples_split=2, n_estimators=33; total time=   0.9s
[CV] END bootstrap=False, criterion=gini, min_samples_leaf=1, min_samples_split=2, n_estimators=13; total time=   0.5s
[CV] END bootstrap=True, criterion=entropy, min_samples_leaf=4, min_samples_split=2, n_estimators=33; t

### Support vector machine

In [ ]:
svm_clf = Classifier('support vector machine', 
	SVC(class_weight='balanced'),
	param={
		'C' : loguniform(1e-2, 1e1),
		'kernel' : ['linear', 'poly', 'rbf'],
		'gamma' : loguniform(1e-2, 1)
	})

svm_clf.optimize_param(ds, random_state=37, verbose=2)
print(svm_clf.get_metrics(ds, as_string=True))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/person

[CV] END C=6.815375811485463, gamma=0.08476105402021752, kernel=linear; total time=   0.5s
[CV] END C=6.815375811485463, gamma=0.08476105402021752, kernel=linear; total time=   0.5s
[CV] END C=6.815375811485463, gamma=0.08476105402021752, kernel=linear; total time=   0.5s
[CV] END C=6.815375811485463, gamma=0.08476105402021752, kernel=linear; total time=   0.5s
[CV] END C=6.815375811485463, gamma=0.08476105402021752, kernel=linear; total time=   0.6s


/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=1.857683598624964, gamma=0.041624068426124705, kernel=rbf; total time=   1.4s


...

[CV] END C=0.40824169553007383, gamma=0.07545256456577856, kernel=rbf; total time=   1.5s
[CV] END C=2.8462452830218363, gamma=0.016462410666268665, kernel=linear; total time=   0.7s
[CV] END C=2.8462452830218363, gamma=0.016462410666268665, kernel=linear; total time=   0.7s
[CV] END C=2.8462452830218363, gamma=0.016462410666268665, kernel=linear; total time=   0.7s


/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=3.9497387084106608, gamma=0.11021542092210415, kernel=poly; total time=   0.9s
[CV] END C=3.9497387084106608, gamma=0.11021542092210415, kernel=poly; total time=   0.9s
[CV] END C=3.9497387084106608, gamma=0.11021542092210415, kernel=poly; total time=   0.8s
[CV] END C=3.9497387084106608, gamma=0.11021542092210415, kernel=poly; total time=   0.9s
[CV] END C=3.9497387084106608, gamma=0.11021542092210415, kernel=poly; total time=   0.8s


/home/jsv/personal/data_mining_sem/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


name: support vector machine, accuracy_train: 0.252, accuracy_test: 0.239, balanced_accuracy_train: 0.499, balanced_accuracy_test: 0.489
